# Lorenz 3d Animation with Matplotlib 

---
## Create Set Of Arbitrary Points to Animate
    I'm going to pretend that I have an engine mounted in four places and
    I want to animate the rigid body motion.

In [29]:
import numpy as np
from scipy import integrate

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

# arbitrary 'mode' frequency, or natural frequency
wn = 10

# arbitrary amount of time the animation should perform
t = np.arange(0,100,0.01)

# arbitary eigenvectors for 6 free-body modes in 6DOF
eigv = [-3.406e-2, -1.654e-1, 1.672e-1, -9.988e-1, -3.823e-1, -7.623e-2]
# eigv = [-4.619e-1,  3.597e-1, -4.013e-4,  1.361e-2, -8.785e-1,  9.680e-1]
# eigv = [-8.862e-1, -9.180e-1, -9.860e-1, -4.679e-2,  2.864e-1,  2.389e-1]
# eigv = [-1.503e-2,  1.845e-3,  1.646e-3,  4.313e-5,  8.095e-4, -1.391e-3]
# eigv = [ 1.968e-3,  3.761e-3, -1.553e-3, -1.562e-3,  8.576e-3,  2.219e-3]
# eigv = [ 3.315e-3,  6.037e-3, -8.172e-3, -6.324e-4, -3.541e-3, -2.326e-3]

# generating inital arbitrary point cloud
eng = [1500, -10, 140]
pt1 = [1700, -250, 15]
pt2 = [1400, -250, 10]
pt3 = [1375, 200, 0]
pt4 = [1725, 200, 25]

p_cld = [eng, pt1, pt2, pt3, pt4]
pts   = np.array(p_cld)



#### Here I plot my point cloud array in a 3d scatter to get an idea of how it looks

In [31]:
# % matplotlib inline
fig = plt.figure()
ax  = fig.add_axes([0, 0, 1, 1], projection='3d')

for pt in pts:
    ax.scatter(pt[0],pt[1],pt[2], s=200, marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

print pts
    

[[1500  -10  140]
 [1700 -250   15]
 [1400 -250   10]
 [1375  200    0]
 [1725  200   25]]


#### Now I must generate the data that governs how these points move

    - Basically I'm only generating data for the 'eng' point.
    - Movement in a specific DOF is dictated by that eigenvalue's
      relationship to the other eigenvalues in the eigenvector 
          - (+/-) for phase relationship
          - magnitude for scale relationship
          
    - After generating the data I will have to transform and rotate the rest
      of the values in the point cloud appropriately

In [122]:
def periodic_motion_gen(eigv, wn, time):
    #I probably want to increase the scale of the movement
    t_sf = 50  # scale factor for translation
    r_sf = .5   # scale factor for rotation
    dx = []    
    for i, eig in enumerate(eigv):
        if i<3:
            dx.append(eig*np.sin(wn*t)*t_sf)
        else:
            dx.append(eig*np.sin(wn*t)*r_sf)
    np.array(dx)
    return np.transpose(dx)

dx = periodic_motion_gen(eigv, wn, t)

print np.shape(dx)
print pts

(10000, 6)
[[1500  -10  140]
 [1700 -250   15]
 [1400 -250   10]
 [1375  200    0]
 [1725  200   25]]


#### I need functions to do the following for a given point in time
    1. Generate the 3D path my CG will move on, since its movement governs
       what all the other points should be doing.
       
    2. Toss new 3D path, to a execute function with 1 pt from the pt cloud
       at a time... 
           - Basically the CG path will be used as a reference to 
             generate the paths of subsequent points
       
   
      

##### #1 Generate 3D Path for CG point

In [123]:
def create_cg_path(cg, dx):
    cg_p = []
    for i, dx_i in enumerate(dx):
        cg += dx_i[:3]
        cg_p.append([cg[0], cg[1], cg[2]])
    return np.asarray(cg_p)

# checker
cg_path = create_cg_path(cg=[1500, -10, 140], dx=dx)



##### #2 Create Paths for all the other points in the point cloud 1X1

In [132]:

def create_pt_paths(cg_path, dx, pt):
    pt_path = []
    
    def trans_pt_by_sig(p, xi, i):
        tpbs = []
        p += dx[i][:3]
        tpbs.append([p[0], p[1], p[2]])
        return np.asarray(tpbs)
    
    
    def trans_pt_2_origin(p, xi, i):
        tp2o = []
        T = np.matrix([[1, 0, 0, -xi[0]],
                       [0, 1, 0, -xi[1]],
                       [0, 0, 1, -xi[2]],
                       [0, 0, 0,     1]])
        
        tmp = np.insert(p,3,1)
        tmp = np.matmul(T,tmp)
        tmp = np.delete(tmp,3)
        tp2o.append(tmp)
        return np.asarray(tp2o)
    
    
    def rotate_pt(p, xi, i):
        rp = []
        a = dx[i][3]
        b = dx[i][4]
        g = dx[i][5]
        
        Rx = np.matrix([[1,         0,          0],
                        [0, np.cos(a), -np.sin(a)],
                        [0, np.sin(a), np.cos(a)]])

        Ry = np.matrix([[ np.cos(b), 0,  np.sin(b)],
                        [        0,  1,          0],
                        [-np.sin(b), 0,  np.cos(b)]])

        Rz = np.matrix([[np.cos(g), -np.sin(g), 0],
                        [np.sin(g),  np.cos(g), 0],
                        [        0,          0, 1]])
        
        p = np.squeeze(p)
        tmp = p
        tmp = np.matmul(tmp,Rx)
        tmp = np.matmul(tmp,Ry)
        tmp = np.matmul(tmp,Rz)
        rp.append(tmp)
        return np.asarray(rp)
    
    def trans_pt_2_cg_instance(p, xi, i):
        tp2ci = []
        T = np.matrix([[1, 0, 0, -xi[0]],
                       [0, 1, 0, -xi[1]],
                       [0, 0, 1, -xi[2]],
                       [0, 0, 0,     1]])
        
        invT = np.linalg.inv(T)
    
        tmp = np.insert(p,3,1)
        tmp = np.matmul(invT, tmp)
        tmp = np.delete(tmp, 3)
    
        tp2ci.append(tmp)
        return np.asarray(tp2ci)
    
    for i, inst in enumerate(cg_path):
        pt_f1 = trans_pt_by_sig(pt, inst, i)
        pt_f2 = trans_pt_2_origin(pt_f1, inst, i)
        pt_f3 = rotate_pt(pt_f2, inst, i)
        pt_f4 = trans_pt_2_cg_instance(pt_f3, inst, i)
        pt_path.append(pt_f4)
        
    return np.asarray(pt_path)

print pts
pt_path = np.asarray([create_pt_paths(cg_path, dx, pt=p) for p in pts])
pt_path = np.squeeze(pt_path)
print np.shape(pt_path)



[[1500  -10  140]
 [1700 -250   15]
 [1400 -250   10]
 [1375  200    0]
 [1725  200   25]]


TypeError: Cannot cast ufunc add output from dtype('float64') to dtype('int32') with casting rule 'same_kind'

In [129]:


% matplotlib qt
fig = plt.figure()
ax  = fig.add_axes([0, 0, 1, 1], projection='3d')


print np.shape(pt_path)


for pt in pt_path:
    plt.plot(pp[0][:100],pp[1][:100],pp[2][:100])


# for i,val in enumerate(cg_path[:100]):
#     ax.scatter(val[0], val[1], val[2])
#     ax.scatter(pt_path[i][0], pt_path[i][1], pt_path[i][2])
    
# pp = np.transpose(pt_path)
# cgp = np.transpose(cg_path)

# plt.plot(pp[0][:100],pp[1][:100],pp[2][:100])
# plt.plot(cgp[0][:100],cgp[1][:100],cgp[2][:100])
    
for pt in pts:
    ax.scatter(pt[0],pt[1],pt[2], s=200, marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()


(5, 10000, 3)
